In [1]:
import os

import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from itertools import chain

import hnswlib

In [2]:
BASE_DIR = "/Users/artemvopilov/Programming/yandex_cup_2023"

In [32]:
DATA_DIR = f"{BASE_DIR}/data"

TRAIN_DF_PATH = f"{DATA_DIR}/train.csv"
TEST_DF_PATH = f"{DATA_DIR}/test.csv"

NORMED_EMBEDDINGS_DIR = f"{BASE_DIR}/normed_embeddings"
PCA_EMBEDDINGS_DIR = f"{BASE_DIR}/pca_embeddings"
VAE_EMBEDDINGS_DIR = f"{BASE_DIR}/vae_embeddings"
NORMED_LSTM_EMBEDDINGS_DIR = f"{BASE_DIR}/normed_lstm_embeddings"

In [38]:
TAGS_N = 256

DIM = 64

### Read data

In [34]:
train_df = pd.read_csv(TRAIN_DF_PATH)
test_df = pd.read_csv(TEST_DF_PATH)

In [35]:
train_track_ids = set(train_df["track"].unique())
test_track_ids = set(test_df["track"].unique())

In [36]:
train_track_id_to_tags = {}
for _, row in tqdm(train_df.iterrows()):
    track_id = row["track"]
    tags = list(map(int, row["tags"].split(",")))
    train_track_id_to_tags[track_id] = tags

0it [00:00, ?it/s]

### Read embeddings

In [37]:
track_id_to_embeddings = {}
for fn in tqdm(os.listdir(VAE_EMBEDDINGS_DIR)):
    fp = f"{VAE_EMBEDDINGS_DIR}/{fn}"

    track_id = fn.split('.')[0]
    embeddings = np.load(fp)
    track_id_to_embeddings[track_id] = embeddings

  0%|          | 0/76714 [00:00<?, ?it/s]

In [39]:
len(track_id_to_embeddings)

76714

In [40]:
track_ids = []
embeddings = []
for ti, embeds in tqdm(track_id_to_embeddings.items()):
    if int(ti) not in train_track_ids:
        continue
    track_ids.append(ti)
    embeddings.append(embeds[-1])

  0%|          | 0/76714 [00:00<?, ?it/s]

In [41]:
len(embeddings)

51134

### KNN index

In [42]:
knn_index = hnswlib.Index(space='cosine', dim=DIM)

In [43]:
knn_index.init_index(len(embeddings))

In [44]:
knn_index.add_items(embeddings)

### Predict

In [45]:
all_track_ids = []
all_embeddings = []
for ti, embeds in tqdm(track_id_to_embeddings.items()):
    all_track_ids.append(ti)
    # all_embeddings.append(np.mean(embeds, axis=0))
    all_embeddings.append(embeds[-1])

  0%|          | 0/76714 [00:00<?, ?it/s]

In [46]:
track_id_to_scores = {}
for ti, embed in tqdm(zip(all_track_ids, all_embeddings)):
    labels, distances = knn_index.knn_query(embed, k=21)

    if ti in train_track_ids:
        labels = labels[0][1:]
        distances = distances[0][1:]
    else:
        labels = labels[0][:-1]
        distances = distances[0][:-1]

    distances_sum = np.sum(distances)
    scores = np.zeros(TAGS_N)
    for l, d in zip(labels, distances):
        track_id = track_ids[l]
        tags = train_track_id_to_tags[int(track_id)]

        target = np.zeros(TAGS_N)
        target[tags] = 1
        scores += target * d / distances_sum
    track_id_to_scores[ti] = scores

0it [00:00, ?it/s]

/var/folders/zx/qc06bh8n1sl80x1x3xp2j5qw0000gn/T/ipykernel_28889/2425898566.py:20: RuntimeWarning: divide by zero encountered in divide
  scores += target * d / distances_sum
/var/folders/zx/qc06bh8n1sl80x1x3xp2j5qw0000gn/T/ipykernel_28889/2425898566.py:20: RuntimeWarning: invalid value encountered in divide
  scores += target * d / distances_sum


### Save scores 

In [47]:
predictions_df = pd.DataFrame([
    {'track': track, 'prediction': ','.join([str(p) for p in probs])}
    for track, probs in track_id_to_scores.items()
])

In [48]:
predictions_df.head()

,track,prediction
0,531,"0.8617021276595747,0.2606382978723404,0.132978..."
1,33632,"0.25,0.5,0.25,0.25,0.0,0.25,0.0,0.0,0.0,0.0,0...."
2,75667,"0.3684979738258155,0.43679000863615225,0.04364..."
3,65474,"0.36363636363636365,0.2727272727272727,0.13636..."
4,23421,"0.3170028818443804,0.6671469740634005,0.097982..."


In [49]:
predictions_df.shape

(76714, 2)

In [50]:
predictions_df.to_csv('prediction_knn_vae_last.csv', index=False)